## Machine learning to create a Trading Algorithim

In this project We are trying to predict the buy and sell price for APPL using ML by using lots of variable that can impact prediction. We are aming to achive a average score that is highet then random guessig.

In [1]:
#Importing Required Libraries

import pandas as pd
import numpy as np
from finta import TA
import matplotlib.pyplot as plt
from pandas.tseries.offsets import DateOffset
import seaborn as sns
import matplotlib.pyplot as plt
import os 
from datetime import datetime, timedelta
from dotenv import load_dotenv
from alpaca_trade_api.rest import REST, TimeFrameUnit, TimeFrame
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report

Setting up API

In [2]:
load_dotenv()
#Set API and secret key
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca API object
api = REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")


ValueError: ('Key ID must be given to access Alpaca trade API', ' (env: APCA_API_KEY_ID)')

Setting API Parameters to download sticker date

In [ ]:
#Selecting ticker
ticker = "AAPL"

#Set timeframe
timeframe = TimeFrame(15, TimeFrameUnit.Minute)

#Setting date for latest date available from APIO
now = pd.Timestamp(datetime.now().date() - timedelta(days=2))

#Caculate the start date 
start_date = (now - timedelta(days=365*10)).date().isoformat()

#Downloading historial data from API
df_ticker = api.get_bars(
    ticker,
    timeframe,
    start=start_date,
).df

# Rename index to "Date"
df_ticker.index.name = 'Date'

In [ ]:
df_ticker.head()

### Generating the Technical Indicators and Custom trading signals by the help of the features.

In [ ]:
indicators_df = pd.DataFrame()

indicators_df['Close'] = df_ticker['close']

# Calculate daily returns
indicators_df['Actual Returns'] = df_ticker['close'].pct_change()

#defining short and long window to caculaute EMA
short_window = 100
long_window = 200

# Caulcualating the EMA technical indicators for the short and long windows
indicators_df["Short"] = TA.EMA(df_ticker, short_window)
indicators_df["Long"] = TA.EMA(df_ticker, long_window)

#Caculuating the RSI indicator
indicators_df["RSI"] = TA.RSI(df_ticker)

# Calculate the upper and lower Bollinger Bands
indicators_df['20 Day MA'] = df_ticker['close'].rolling(window=20).mean()
indicators_df['Upper Band'] = indicators_df['20 Day MA'] + 2 * df_ticker['close'].rolling(window=20).std()
indicators_df['Lower Band'] = indicators_df['20 Day MA'] - 2 * df_ticker['close'].rolling(window=20).std()


In [ ]:
indicators_df.head()

#### Finding the correalation between the variables

Here we are performing the correlation matrix and visualizing them on a heat map to figure out whcih variables are less correlated.

If the correaltion:
- Is between 0.0 and -0.3, they have a week correlation.
- Is between -0.3 and -0.5 they have a moderate correlation.
- Is between 0.5 and -1 they have a high correlation.
- is between 0.0 and 0.3 they have a week correlation.
- Is between 0.3 and 0.5 they have a moderate correlation.
- Is between0.5 and 1 they have a high correlation.

In [ ]:
# Generate the correlation matrix
corr_matrix = indicators_df.dropna().corr()

# Generate the heatmap
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(
    corr_matrix,
    annot=True, 
    cmap='coolwarm',
    ax=ax
    )

In [ ]:
# Trading Rules - buy, hold, and sell signals based on the RSI indicator
for i, row in indicators_df.iterrows():
    # Buy Signal
    if indicators_df.loc[i, 'RSI'] < 30:
        indicators_df.loc[i, 'Buy Signal'] = 1
    # Sell Signal
    elif indicators_df.loc[i, 'RSI'] > 70:
        indicators_df.loc[i, 'Buy Signal'] = -1
    else:
        # Hold
        indicators_df.loc[i, 'Buy Signal'] = 0


In [ ]:
#Count buy, sell and hold values
indicators_df['Buy Signal'].value_counts()

In [ ]:
# Visualise Backtest Peformance

# Plot the Close price and the Bollinger Bands
plt.figure(figsize=(20,10))
plt.plot(indicators_df['Close'])
plt.plot(indicators_df['RSI'])

# Plot the signals
plt.plot(indicators_df[indicators_df['Buy Signal'] == 1.0].index, 
         indicators_df['Close'][indicators_df['Buy Signal'] == 1.0],
         '^', markersize=1, color='green', label='Buy')
plt.plot(indicators_df[indicators_df['Buy Signal'] == -1.0].index, 
         indicators_df['Close'][indicators_df['Buy Signal'] == -1.0], 
         'v', markersize=1, color='red', label='Sell')

plt.title('Stock Prices with RSI and Trading Signals')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()


Theoretical Stock Price Predictions - Caculating return. 

In [ ]:
# Create a new DataFrame with only the 'Close' column
appl_df = df_ticker[['close']].copy()

# Reindex using business days frequency
appl_df = appl_df.reindex(pd.bdate_range(start=df_ticker.index.min().date(), end=datetime.now().date(), freq='B'))

# Review the DataFrame
appl_df.tail()


In [ ]:
# Initialize trade_type column for buys and sells
appl_df['trade_type'] = np.nan

# Initialize variable to hold the previous_price
previous_price = 0

# Initialize a cost/proceeds column for recording trade metrics
appl_df["cost/proceeds"] = np.nan


In [ ]:
# Initialize share_size equals 100 and accumulated_shares equals 0
share_size = 100
accumulated_shares = 0


In [ ]:
# Loop through the Pandas DataFrame and code the conditions of the trading strategy
for index, row in appl_df.iterrows():

    # buy if the previous price is 0, in other words, buy on the first day
    # set the cost/proceeds column equal to the negative value of the row close price
    # multiplied by the share_size
    if previous_price == 0:
        if indicators_df.loc[index, 'RSI'] < 30:
            appl_df.loc[index, "cost/proceeds"] = -(row["close"] * share_size)
            accumulated_shares += share_size
            print(f"{index}: Bought {share_size} shares, total {accumulated_shares}")

    # buy if the current day price is less than the previous day price
    # set the cost/proceeds column equal to the negative value of the row close price
    # multiplied by the share_size
    elif row["close"] < previous_price:
        if indicators_df.loc[index, 'RSI'] < 30:
            appl_df.loc[index, "cost/proceeds"] = -(row["close"] * share_size)
            accumulated_shares += share_size
            print(f"{index}: Bought {share_size} shares, total {accumulated_shares}")

    # sell if the current day price is greater than the previous day price
    elif row["close"] > previous_price and accumulated_shares > 0 and index != appl_df.index[-1]:
        if indicators_df.loc[index, 'RSI'] > 70:
            appl_df.loc[index, "cost/proceeds"] = row["close"] * share_size
            print(f"{index}: Sold {accumulated_shares} shares")
            accumulated_shares -= share_size
            # accumulated_shares = 0

    # else hold if the current day price is equal to the previous day price
    else:
        appl_df.loc[index, "trade_type"] = "hold"

    # set the previous_price variable to the close price of the current row
    previous_price = row["close"]

    # if the index is the last index of the Dataframe, sell
    # set the cost/proceeds column equal to the row close price multiplied
    # by the accumulated_shares
    if index == appl_df.index[-1]:
        appl_df.loc[index, "trade_type"] = "sell"
        appl_df.loc[index, "cost/proceeds"] = row["close"] * accumulated_shares
        print(f"{index}: Sold {accumulated_shares} shares")
        accumulated_shares = 0


In [ ]:
# Calculate the return on investment (ROI)
roi = round((total_profit_loss / -(invested_capital)) * 100, 2)

# Print the ROI
print(f"The trading algorithm resulted in a return on investment of {roi}%")


In [ ]:
indicators_df['Strategy Returns'] = indicators_df['Actual Returns'] * indicators_df['Buy Signal'].shift()
indicators_df = indicators_df.dropna()

display(indicators_df.head())
display(indicators_df.tail())

In [ ]:
(1 + indicators_df['Strategy Returns']).cumprod().plot()


In [ ]:
# Assign a copy of the RSI columns to a features DataFrame called X 
X = indicators_df[['RSI']].shift().dropna()

# Assign Buy Signals columns to a features DataFrame called Y
y = indicators_df['Buy Signal']
display(X.head())
display(y.value_counts())

### Split the data into training and testing using time series split

In [ ]:
# Select the start of the training period
training_begin = X.index.min()

# Select the ending period for the training data with an offset of 12 months
training_end = X.index.min() + DateOffset(months=12)


In [ ]:
# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

# Review the X_train DataFrame
X_train.head()

In [ ]:
# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end+DateOffset(hours=1):]
y_test = y.loc[training_end+DateOffset(hours=1):]

# Review the X_test DataFrame
X_test.head()

Scacling the Data

In [ ]:

# Scale the features DataFrames

# Create a StandardScaler instance
scaler = StandardScaler()

# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)

# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

As we want to predictBuy/sell Signal, we will go ahead with the classification approach and use SVC classifier from SKLearns Ssupport vector machine (SVM) learning method o fit the training data and make predictions based on the testing data.

In [ ]:
# From SVM, instantiate SVC classifier model instance
svm_model = SVC()
 
# Fit the model to the data using the training data
svm_model = svm_model.fit(X_train_scaled, y_train)
 
# Use the testing data to make the model predictions
svm_pred = svm_model.predict(X_train_scaled)

# Review the model's predicted values
print(svm_pred)

#### Review the classification report associated with the `SVC` model predictions. 

In [ ]:
# Use a classification report to evaluate the model using the predictions and testing data
svm_testing_report = classification_report(y_train, svm_pred)

# Print the classification report
print(svm_testing_report)

#### Create a predictions DataFrame that contains columns for “Predicted” values, “Actual Returns”, and “Strategy Returns”.

In [ ]:
# Create a new empty predictions DataFrame:

# Create a predictions DataFrame
predictions_df = pd.DataFrame(index=y_train.index)

# Add the SVM model predictions to the DataFrame
predictions_df['Predicted'] = svm_pred

# Add the actual returns to the DataFrame
predictions_df['Actual Returns'] = y_train

# Add the strategy returns to the DataFrame
predictions_df['Strategy Returns'] = predictions_df['Actual Returns'] * predictions_df['Predicted']

# Review the DataFrame
display(predictions_df.head())
display(predictions_df.tail())

#### Create a cumulative return plot that shows the actual returns vs. the strategy returns. Save a PNG image of this plot. This will serve as a baseline against which to compare the effects of tuning the trading algorithm.

In [ ]:
# Plot the actual returns versus the strategy returns
((1 + predictions_df[['Actual Returns', 'Strategy Returns']]
  .cumsum())
 .plot(figsize=(15, 8)))
plt.title('Actual vs. Strategy Returns')
plt.show()

plt.savefig('Actual vs. Strategy Returns.png')

#### Backtesting a Machine Learning Trading Algorithm

In [ ]:
# Use the trained model to predict the trading signals for the testing data.
testing_signal_predictions = svm_model.predict(X_test_scaled)

In [ ]:
# Evaluate the model's ability to predict the trading signal for the testing data
testing_report = classification_report(y_test, testing_signal_predictions)

# Display the report
print(testing_report)

#### Create a predictions DataFrame from back testing that contains columns for “Predicted” values, “Actual Returns”, and “Strategy Returns”.

In [ ]:
# Create a new empty predictions DataFrame:

# Create a predictions DataFrame
backtest_predictions_df = pd.DataFrame(index=y_test.index)

# Add the SVM model predictions to the DataFrame
backtest_predictions_df['Predicted'] = testing_signal_predictions

# Add the actual returns to the DataFrame
backtest_predictions_df['Actual Returns'] = y_test

# Add the strategy returns to the DataFrame
backtest_predictions_df['Strategy Returns'] = backtest_predictions_df['Actual Returns'] * backtest_predictions_df['Predicted']

# Review the DataFrame
display(backtest_predictions_df.head())
display(backtest_predictions_df.tail())

#### Create a cumulative return plot that shows the backtested actual returns vs. the strategy returns. Save a PNG image of this plot. This will serve as a baseline against which to compare the effects of tuning the trading algorithm.

In [ ]:
# Plot the actual returns versus the strategy returns
((1 + backtest_predictions_df[['Actual Returns', 'Strategy Returns']]
  .cumsum())
 .plot(figsize=(15, 8)))
plt.title('Back Tested Actual vs. Strategy Returns')
plt.show()

# plt.savefig('Back Tested Actual vs. Strategy Returns')

### Tune the Baseline Trading Algorithm Using Alternative ML Model - I am still working on This Asha I have not finished.

In this section, you’ll tune, or adjust, the model’s input features to find the parameters that result in the best trading outcomes. You’ll choose the best by comparing the cumulative products of the strategy returns.

In [ ]:

# # Define the period for training and testing datasets
# training_period = 300 # number of days
# testing_period = 70 # number of days

In [ ]:
# # Loop through the time range in increments of testing period
# for i in range(training_period, len(X_train) - testing_period, testing_period):
    
#     # Define the start and end dates for the training and testing data
#     start_train_date = X_train.index[i - training_period]
#     end_train_date = X_train.index[i - 1]
#     start_test_date = X_test.index[i]
#     end_test_date = X_test.index[i + testing_period - 1]

In [ ]:
    # # Slice the data into the training and testing datasets
    # train_df = X_train.loc[start_train_date:end_train_date]
    # test_df = X_test.loc[start_test_date:end_test_date]

In [ ]:
    # # Define the input features and target variable for the AdaBoost model
    # X_train = train_df[['SMA_Fast', 'SMA_Slow']].shift()
    # y_train = signals_df['Signal'].shift()
    # X_test = test_df[['SMA_Fast', 'SMA_Slow']].shift()
    # y_test = signals_df['Signal'].shift()

### Using Alternative mode to see if we can achive higher average score

In [ ]:
# Instantiate the model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Fit the model to the training data
rf_model.fit(X_train_scaled, y_train)

# Use the model to make predictions on the testing data
rf_pred = rf_model.predict(X_train_scaled)

print(rf_pred)

In [ ]:
# Use a classification report to evaluate the model using the predictions and testing data
rf_testing_report_train = classification_report(y_train, rf_pred)

# Print the classification report
print(rf_testing_report_train)

In [ ]:
# Use the trained model to predict the trading signals for the testing data.
testing_signal_predictions = rf_model.predict(X_test_scaled)

In [ ]:
# Evaluate the model's ability to predict the trading signal for the testing data
testing_report_test = classification_report(y_test, testing_signal_predictions)

# Display the report
print(testing_report_test)

Adaboose Model  

In [ ]:
# Initiate the model instance
adaboost_model = AdaBoostClassifier()

# Fit the model using the training data
model = adaboost_model.fit(X_train_scaled, y_train)

# Use the testing dataset to generate the predictions for the new model
pred = adaboost_model.predict(X_test_scaled)

# Review the model's predicted values
print(pred)


In [ ]:
# Use a classification report to evaluate the model using the predictions and testing data
ada_pred = classification_report(y_test, pred)

# Print the classification report
print(ada_pred)


In [ ]:
# Create a new empty predictions DataFrame:
# Create a predictions DataFrame
ada_predictions_df = pd.DataFrame(index=X_test.index)

# Add the SVM model predictions to the DataFrame
ada_predictions_df['Predicted'] = pred

# Add the actual returns to the DataFrame
ada_predictions_df['Actual Returns'] = indicators_df['Actual Returns']

# Add the strategy returns to the DataFrame
ada_predictions_df['Strategy Returns'] = ada_predictions_df['Actual Returns'] * pred

# Review the DataFrame
ada_predictions_df


In [ ]:
# Plot the actual returns versus the strategy returns
(1 + ada_predictions_df[['Actual Returns', 'Strategy Returns']]).cumprod().plot()